## Mapping tool for internal use
#### - Maps point geometries
#### - Builds choropleth maps of the frequency of those points within the boundaries of polygon geometries
#### - Includes a reference layer showing the number of points per 1000 population

This tool is designed for internal use to quickly highlight areas of interest for further analysis. 
It should be easy to drop data in and get something out!
Because it doesn't assume that the geographies it's being fed have a standard output area code, the program uses spatial joins rather than NSUL lookups. As this means it may be slightly inaccurate especially around borders it shouldn't be relied on to produce public-facing statistics.

By default it produces maps using the quartile scheme with 10 bins. This behaviour can be changed in the options in cell 2.

##### By: Samantha Iacob (samantha.iacob@ons.gov.uk)

Last edited: 25/6/2024

To do: 

- Panel controls to allow quick adjusting what the choropleth maps are showing (longer term)

#### Run cells 1-4 in order. Cells 5 and 6 are optional

Cell 1: Include variables (edit the values within the quotations to the paths to your .shp files within the
data folder. Please fill in the other details as well.

In [ ]:
#file variables (important)
point_data_file = r"post_offices_and_post_boxes_EW.csv" #shp or csv file in data directory, converts csv to point geometry if needed
point_data_classification_code_column_name = "class_code" #shouldn't need to change this normally
point_data_classification_description_column_name = "class_desc" #shouldn't need to change this normally

# polygon_data_file_1 should be the largest geography as it will set the default zoom

polygon_data_file_1 = r"MSOA_2021_EW_BGC_V2_493701222045311232\MSOA_2021_EW_BGC_V2.shp"
polygon_data_file_1_description = "England & Wales: MSOAs 2021"
polygon_data_file_1_short_description = "England & Wales"
polygon_data_file_1_OA_code = "MSOA21CD"
polygon_data_file_1_OA_name_override = "" #optional, this value is calculated from the OA code if left blank

polygon_data_file_2 = r""
polygon_data_file_2_description = ""
polygon_data_file_2_short_description = ""
polygon_data_file_2_OA_code = ""
polygon_data_file_2_OA_name_override = "" #optional, this value is calculated from the OA code if left blank

polygon_data_file_3 = r""
polygon_data_file_3_description = ""
polygon_data_file_3_short_description = ""
polygon_data_file_3_OA_code = ""
polygon_data_file_3_OA_name_override = "" #optional, this value is calculated from the OA code if left blank

polygon_data_file_4 = r""
polygon_data_file_4_description = ""
polygon_data_file_4_short_description = ""
polygon_data_file_4_OA_code = ""
polygon_data_file_4_OA_name_override = "" #optional, this value is calculated from the OA code if left blank

Cell 2: Optional settings, please adjust to your liking if you feel like it.

In [ ]:
#options (change if needed)

#Set this to false to not generate a comparison map showing population based on 2021 LA boundaries. This saves a
#little time and space if not needed
LA_comparison = True

#Save csv point data (if present) to a shapefile in output folder. Set to False to prevent this.
save_point_shapefile = True

#the colour of each classification's points on the map will be drawn from this list in order.
#the colours were taken from here https://sashamaps.net/docs/resources/20-colors/ using the 99% accessible version
#and removing red, green and yellow as they don't show up well against the colour maps I picked.

colour_list = ['#4363d8',
 '#f032e6',
 '#469990',
 '#000075',
 '#800000',
 '#42d4f4',
 '#a9a9a9',
 '#dcbeff',
 '#911eb4',
 '#aaffc3',
 '#9A6324',
 '#fffac8',
 '#f58231',
 '#000000',
 '#ffffff']

#The scheme and bin count used for the polygon geometry choropleth maps. 
#Available schemes : 
'''
'boxplot', 'equalinterval', 'fisherjenks', 'fisherjenkssampled', 
'headtailbreaks', 'jenkscaspall', 'jenkscaspallforced', 'jenkscaspallsampled', 
'maxp', 'maximumbreaks', 'naturalbreaks', 'quantiles', 'percentiles', 'stdmean', 'userdefined'
'''
scheme = 'quantiles'
k = 10

#the colour map used for each polygon geometry set on the map. 
#https://matplotlib.org/stable/gallery/color/colormap_reference.html for more information
colormaps = ['summer_r','autumn_r','winter_r','spring_r']

Cell 3: The code that prepares the data for the map. Please run it but don't alter things in here unless necessary

In [ ]:
#libraries
import os
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import GroupedLayerControl
from datetime import datetime
from sys import exit

#time and savefile variables
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
x = 1

#functions for this tool
def point_data_from_csv(point_data_file = point_data_file, timestamp = timestamp, point_data_classification_code_column_name = "class_code"):
    '''
    merges csv output from a SQL query with a classifications list then converts it to a geodataframe containing point
    geometry based on the x and y columns in the data, and then saves it as a shape file in the outputs directory for future use.
    This is only appropriate for UK as x and y are Easting and Northing values and are mapped to the EPSG:27700 CRS.
    '''
    #ingesting data
    point_df = pd.read_csv('data/' + point_data_file)
    classifications_df = pd.read_csv('data/addressbase-product-classification-scheme.csv')
    #merging and arranging dataframes
    point_classified_df = pd.merge(point_df, classifications_df, left_on = point_data_classification_code_column_name, right_on = "Concatenated", how = 'left', copy = False)
    point_classified_df = point_classified_df[["uprn", "class_code", "Class_Desc","x", "y"]]
    point_classified_df.rename(columns = {'Class_Desc': 'class_desc'}, inplace = True)
    #converting merged dataframe to geodataframe
    point_gdf = gpd.GeoDataFrame(point_classified_df, geometry = gpd.points_from_xy(point_classified_df.x, point_classified_df.y), crs = 'EPSG:27700')
    if save_point_shapefile == True:
        #exporting savefile
        os.makedirs(f'outputs/{point_data_file[:-4]}_{timestamp}')
        outfilepath = os.path.join('outputs',f'{point_data_file[:-4]}_{timestamp}/{point_data_file[:-4]}_{timestamp}.shp')
        point_gdf.to_file(outfilepath)
        print(f"{point_data_file} shapefile saved to: outputs/{point_data_file[:-4]}_{timestamp}/{point_data_file[:-4]}_{timestamp}.shp")
    else:
        print(f"{point_data_file} shapefile not saved, set save_point_shapefile to True in Cell 2 to change this behaviour.")
    return point_gdf

def add_aggregate_data_cols(polygon_data, point_data, output_area_col_name, classification_code_col_name = point_data_classification_code_column_name):
    #inputs : a gdf with polygon geometry, a gdf with points geometry - intersecting, the names of the OA and classification code columns.
    #Returns a geodataframe with polygon geometry and with counts of how many of each classification appears within those polygons.
    point_polygon = gpd.sjoin(polygon_data, point_data, how = 'inner', predicate = 'intersects')
    agg_df = point_polygon.groupby(by = [output_area_col_name, classification_code_col_name]).agg('count').reset_index()
    
    area_list = agg_df[output_area_col_name].unique().tolist()
    class_list = agg_df[classification_code_col_name].unique().tolist()
    
    
    new_df = pd.DataFrame(data = {output_area_col_name : area_list} )
    
    #creating a list of dataframes which consist of columns: OA code and count. one for each classification code in the point data 
    frame_of_frames = []
    counter = 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
    for classification in class_list:
        frame_of_frames.append(pd.merge(new_df, agg_df[agg_df[classification_code_col_name] == classification]))
        frame_of_frames[counter].rename(columns = {'geometry' : classification}, inplace = True)
        frame_of_frames[counter] = frame_of_frames[counter][[output_area_col_name, classification]]
        counter +=1
   
    #smushing it together into a single df, and removing NANs                 
    for frame in frame_of_frames:
        new_df = pd.merge(new_df, frame, how = "left", copy = False)
    new_df = new_df.fillna(0) 

    
    #fixing dtypes and populating a total column
    new_df['all'] = 0
    for classification in class_list:
        new_df[classification] = new_df[classification].astype('int32')
        new_df['all'] += new_df[classification]
        
    #merging the classification counts with the dataframe that holds the polygon geometry
    merged_gdf = pd.merge(polygon_data, new_df, how = 'left', on = output_area_col_name, copy = False)
    return merged_gdf

def classification_dictionary(point_data, classification_code_col_name = point_data_classification_code_column_name, class_description_code = point_data_classification_description_column_name):
    #outputs a dictionary of classifications found in the point data gdf
    class_list = classification_list(point_data, classification_code_col_name)
    new_dict = {}
    for item in class_list:
        new_dict[item] = point_data[(point_data[classification_code_col_name] == item)][class_description_code].values[0]
    return new_dict

def classification_list(point_data, classification_code_col_name = point_data_classification_code_column_name):
    #returns a list of unique classifications in the point data
    new_list = point_data[classification_code_col_name].unique().tolist()
    return new_list

def classification_colour(classification_list):
    #returns a dict of point data classifications and an associated colour for mapping
        new_dict = {}
        counter = 0
        for item in classification_list:
            new_dict[item] = colour_list[counter]
            counter += 1
        return new_dict
    
#importing data

if (point_data_file[len(point_data_file)-3:]) == 'csv':
    point_data = point_data_from_csv()
elif (point_data_file[len(point_data_file)-3:]) == 'shp':
    point_data = gpd.read_file(f"data/{point_data_file}")
else:
    exit("Invalid file type for point geometry. Must be .csv or .shp file located within the data directory of this tool.")

#as there can be more than one polygon dataset (england and wales geogs, etc, putting polygon data into a list of dfs)

polygon_data = []
polygon_data_desc = []
polygon_data_short_desc = []
polygon_OA_code = []
polygon_OA_name = []

if polygon_data_file_1 != "":
    polygon_data.append(gpd.read_file(f"data/{polygon_data_file_1}"))
    polygon_data_desc.append(polygon_data_file_1_description)
    polygon_data_short_desc.append(polygon_data_file_1_short_description)
    polygon_OA_code.append(polygon_data_file_1_OA_code)
    if polygon_data_file_1_OA_name_override != "":
        polygon_OA_name.append(polygon_data_file_1_OA_name_override)
    else:
        polygon_OA_name.append(polygon_data_file_1_OA_code[:-2] + "NM")
        
if polygon_data_file_2 != "":
    polygon_data.append(gpd.read_file(f"data/{polygon_data_file_2}"))
    polygon_data_desc.append(polygon_data_file_2_description)
    polygon_data_short_desc.append(polygon_data_file_2_short_description)
    polygon_OA_code.append(polygon_data_file_2_OA_code)
    if polygon_data_file_2_OA_name_override != "":
        polygon_OA_name.append(polygon_data_file_2_OA_name_override)
    else:
        polygon_OA_name.append(polygon_data_file_2_OA_code[:-2] + "NM")
        
if polygon_data_file_3 != "":
    polygon_data.append(gpd.read_file(f"data/{polygon_data_file_3}"))
    polygon_data_desc.append(polygon_data_file_3_description)
    polygon_data_short_desc.append(polygon_data_file_3_short_description)
    polygon_OA_code.append(polygon_data_file_3_OA_code)
    if polygon_data_file_3_OA_name_override != "":
        polygon_OA_name.append(polygon_data_file_3_OA_name_override)
    else:
        polygon_OA_name.append(polygon_data_file_3_OA_code[:-2] + "NM")

if polygon_data_file_4 != "":
    polygon_data.append(gpd.read_file(f"data/{polygon_data_file_4}"))
    polygon_data_desc.append(polygon_data_file_4_description)
    polygon_data_short_desc.append(polygon_data_file_4_short_description)
    polygon_OA_code.append(polygon_data_file_4_OA_code)
    if polygon_data_file_4_OA_name_override != "":
        polygon_OA_name.append(polygon_data_file_4_OA_name_override)
    else:
        polygon_OA_name.append(polygon_data_file_4_OA_code[:-2] + "NM")       

counter = 0
for map in polygon_data:
    polygon_data[counter] = add_aggregate_data_cols(polygon_data[counter], point_data,polygon_OA_code[counter])
    counter +=1

#populating classifications lists and dictionaries.
class_list = classification_list(point_data)
class_dict = classification_dictionary(point_data)
colour_dict = classification_colour(class_list)

#generating a population dataset of LAs which can be used as a comparator layer in the map
#this information is found on the MYE3 tab of the local authority boundary edition of the mid 2021 population estimate
# https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/estimatesofthepopulationforenglandandwales
if LA_comparison == True:
    
    #ingesting data
    res_data = gpd.read_file("data/Local_Authority_Districts_December_2021_GB_BGC_2022_7766355490887282253/LAD_DEC_2021_GB_BGC.shp")
    pop_data = pd.read_csv("data/population_e_w_2021.csv")
    res_data = pd.merge(res_data, pop_data, how = 'left', left_on = "LAD21CD",right_on = "Code", copy = False)[["LAD21CD", "LAD21NM", "BNG_E", "BNG_N","LONG", "LAT", "GlobalID", "geometry","Population"]]
    res_data = res_data[~res_data["LAD21CD"].str.startswith("S")] # removing Scotland. 
    res_data = add_aggregate_data_cols(res_data, point_data, "LAD21CD")
    res_data['per_thousand'] = round((res_data['all']/(res_data['Population'])*1000),3)

Cell 4: run to generate a folium map in the notebook.

In [ ]:
#note: depending on the complexity of the map it can take a short while to load, please be patient.

#this is the target column for the cloropleth maps. If panel controls added later can be adjusted via the controls:
column = 'all'

#for styles to remain consistent between map elements the variables can be edited here

poly_style_kwds = {"fillOpacity": 0.4,}
poly_highlight_kwds = {"fillOpacity" : 0.5,}

marker_style_kwds = {"stroke": True,
                     "opacity" : 1,
                     "fillOpacity" : 1,
                    }
marker_kwds = {"fill" : True,
                "radius": 50,
                }

#generating the tooltips and aliases for the polygon geometry maps
tooltips = []
alias_list = []
counter = 0
for map in polygon_data:
    tooltips.append([polygon_OA_name[counter], polygon_OA_code[counter]])
    alias_list.append([polygon_OA_name[counter], polygon_OA_code[counter]])
    for item in class_list:
        tooltips[counter].append(item)
        alias_list[counter].append(class_dict[item])
    tooltips[counter].append('all')
    alias_list[counter].append('All classifications')
    counter += 1


#rendering the largest geography first as it will set default zoom

counter = 0
for map in polygon_data:
    if counter == 0:
        m = polygon_data[counter].explore(column,
                                          tiles = None,
                                          name = polygon_data_desc[counter],
                                          cmap = colormaps[counter],
                                          scheme = scheme,
                                          k = k,
                                          style_kwds = poly_style_kwds,
                                          highlight_kwds = poly_highlight_kwds,
                                          legend_kwds = {"caption" : f"{polygon_data_short_desc[counter]}, {column} classifications",
                                                        "scale" : False,
                                                        },
                                          tooltip = tooltips[counter],
                                          tooltip_kwds = {"aliases" : alias_list[counter]},
                                         )
    else:
        m = polygon_data[counter].explore(column,
                                          name = polygon_data_desc[counter],
                                          cmap = colormaps[counter],
                                          scheme = scheme,
                                          k = k,
                                          style_kwds = poly_style_kwds,
                                          highlight_kwds = poly_highlight_kwds,
                                          legend_kwds = {"caption" : f"{polygon_data_short_desc[counter]}, {column} classifications",
                                                        "scale" : False,
                                                        },
                                          tooltip = tooltips[counter],
                                          tooltip_kwds = {"aliases" : alias_list[counter]},
                                          m = m,
                                          )
    counter += 1
    
#population data (could be included with all maps using this tool - population as of 2021 using Census 21 LA bounds)

if LA_comparison == True:
    res_alias_list = ['LAD21NM', 'LAD21CD', 'Population']
    res_tooltips= ['LAD21NM', 'LAD21CD', 'Population']
    for item in class_list:
        res_tooltips.append(item)
        res_alias_list.append(class_dict[item])
    res_tooltips.append('all')
    res_tooltips.append('per_thousand')
    res_alias_list.append('All classifications')
    res_alias_list.append('All classifications per 1000 population')
    
    m = res_data.explore("per_thousand",
                     name = "All classifications per 1000 ppl/LA, 2021", 
                     cmap = 'inferno_r', 
                     scheme = 'quantiles',
                     k = 10,
                     style_kwds = {"fillOpacity": 0.9,},
                     highlight_kwds = {"fillOpacity": 1.0,},
                     legend_kwds = {"caption" : "Observations per 1000 population, mid-2021",
                                                        "scale" : False,
                                                        },
                     tooltip = res_tooltips,
                     tooltip_kwds = {"aliases" : res_alias_list},
                     show = False,
                     m = m,
                    )

#point data

for classification in class_list:
    point_data[point_data[point_data_classification_code_column_name] == classification].explore(
        m=m,
        name = class_dict[classification],
        marker_type = "circle",
        marker_kwds = marker_kwds,
        style_kwds = marker_style_kwds,
        color = colour_dict[classification],
    )

# adding selectable tilesets to the map. The last layer here will be the active one when the map is loaded

folium.TileLayer('cartodbdarkmatter', name = 'Tiles: CartoDB Dark Matter').add_to(folium.FeatureGroup('tiles')).add_to(m)
folium.TileLayer('cartodbpositron', name = 'Tiles: CartoDB Positron').add_to(folium.FeatureGroup('tiles')).add_to(m)

#adding a control box element to the map 

folium.LayerControl().add_to(m)

m #display the map

Cell 5: Run to show the map in a tab of your browser window. It will lock the tool while that window is open.

In [ ]:
m.show_in_browser()

Cell 6: This will save your map as a html file in the 'outputs' directory of this tool. It can be opened in your browser.

In [ ]:
m.save(f'outputs/{timestamp}_map_{x}.html')
print(f"map saved to: outputs/{timestamp}_map_{x}.html")
x += 1